<a href="https://colab.research.google.com/github/jessica8403/TIL/blob/master/%EC%9D%B8%EA%B5%AC%EB%8C%80%EB%B9%84%EC%86%8C%EB%93%9D%EA%B8%B0%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 29 02:27:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |    375MiB / 16280MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [5]:
import pandas as pd  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# csv 형식으로 된 데이터 파일을 읽어옵니다.
train = pd.read_csv('/content/drive/MyDrive/깃허브/인구데이터기반소득예측/train.csv')

train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
1,1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0
4,4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0


In [6]:
train.shape

(17480, 16)

In [7]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 있는 컬럼은: workclass 입니다
해당 컬럼에 총 1836 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: occupation 입니다
해당 컬럼에 총 1843 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: native.country 입니다
해당 컬럼에 총 583 개의 결측치가 존재합니다.


In [8]:
print(train['workclass'].unique())
print(train['occupation'].unique())
print(train['native.country'].unique())

['Private' 'State-gov' 'Local-gov' 'Self-emp-not-inc' 'Self-emp-inc'
 'Federal-gov' 'Without-pay' nan 'Never-worked']
['Craft-repair' 'Exec-managerial' 'Adm-clerical' 'Prof-specialty'
 'Machine-op-inspct' 'Other-service' 'Sales' 'Farming-fishing'
 'Transport-moving' 'Handlers-cleaners' 'Tech-support' 'Protective-serv'
 'Priv-house-serv' 'Armed-Forces' nan]
['United-States' 'Poland' 'Mexico' 'Ireland' 'Guatemala'
 'Dominican-Republic' 'Greece' 'El-Salvador' 'Portugal' 'Canada'
 'Philippines' 'India' 'Italy' 'England' 'Jamaica' 'Columbia' 'South'
 'Vietnam' 'Cuba' 'Laos' 'Hong' 'Haiti' 'Germany' 'Yugoslavia' 'Ecuador'
 'France' 'Puerto-Rico' 'Outlying-US(Guam-USVI-etc)' 'Taiwan' 'China'
 'Japan' 'Honduras' 'Peru' 'Nicaragua' 'Hungary' 'Cambodia' 'Iran'
 'Trinadad&Tobago' 'Thailand' 'Scotland' 'Holand-Netherlands' nan]


In [9]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 범주형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
    return temp

train = handle_na(train, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(train) 

결측치가 존재하지 않습니다


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15081 entries, 0 to 15080
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              15081 non-null  int64 
 1   age             15081 non-null  int64 
 2   workclass       15081 non-null  object
 3   fnlwgt          15081 non-null  int64 
 4   education       15081 non-null  object
 5   education.num   15081 non-null  int64 
 6   marital.status  15081 non-null  object
 7   occupation      15081 non-null  object
 8   relationship    15081 non-null  object
 9   race            15081 non-null  object
 10  sex             15081 non-null  object
 11  capital.gain    15081 non-null  int64 
 12  capital.loss    15081 non-null  int64 
 13  hours.per.week  15081 non-null  int64 
 14  native.country  15081 non-null  object
 15  target          15081 non-null  int64 
dtypes: int64(8), object(8)
memory usage: 2.0+ MB


In [11]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            #dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

train = label_encoder(train, make_label_map(train))

In [12]:
train.shape

(15081, 16)

In [13]:
train

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,0,32,0,309513,0,12,0,0,0,0,0,0,0,40,0,0
1,1,33,0,205469,1,10,0,1,0,0,0,0,0,40,0,1
2,2,46,0,149949,1,10,0,0,0,0,0,0,0,40,0,0
3,3,23,0,193090,2,13,1,2,1,0,1,0,0,30,0,0
4,4,55,0,60193,3,9,2,2,2,0,1,0,0,40,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,15076,35,0,337286,7,14,1,1,2,2,0,0,0,40,0,0
15077,15077,36,0,182074,1,10,2,2,2,0,0,0,0,45,0,0
15078,15078,50,4,175070,4,15,0,3,0,0,0,0,0,45,0,1
15079,15079,39,0,202937,1,10,2,10,2,0,1,0,0,40,1,0


In [21]:
from sklearn.model_selection import train_test_split

data = train.drop('id', axis = 1).copy() #필요없는 id열 삭제
train_data, val_data = train_test_split(data, test_size=0.2)
train_data.reset_index(inplace=True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
val_data.reset_index(inplace=True)

In [22]:
print( 'train 데이터 셋 모양 :', train_data.shape)
print( 'val 데이터 셋 모양 :', val_data.shape)

train 데이터 셋 모양 : (12064, 16)
val 데이터 셋 모양 : (3017, 16)


In [23]:
train_data.head()

,index,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,3036,30,0,348592,3,9,0,4,0,0,0,0,0,44,0,0
1,7895,46,0,59287,1,10,0,0,0,0,0,0,0,40,0,0
2,10089,63,0,246841,3,9,0,9,0,0,0,0,0,40,0,0
3,13854,45,0,198759,12,16,0,1,0,0,0,0,0,60,0,0
4,1677,46,0,35406,8,4,3,5,2,0,1,0,0,32,0,0


In [24]:
X_train = train_data.drop(['index', 'target'], axis=1) #training 데이터에서 독립변수 추출
y_train = train_data.target #training 데이터에서 라벨 추출

In [25]:
X_train.shape

(12064, 14)

In [26]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier() # 모델을 객체에 할당
model.fit(X_train, y_train) # 모델 학습

#pred = model.predict(test)

RandomForestClassifier()

In [27]:
X_val = val_data.drop(['index', 'target'], axis=1)  #validation 데이터에서 전처리된 문서 추출
y_val = val_data.target #validation 데이터에서 라벨 추출

In [28]:
y_pred = model.predict(X_val)
print(y_pred)

[0 0 0 ... 1 0 1]


In [29]:
from sklearn import metrics

print('RandomForestClassifier 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred),3)) # 정확도 확인

RandomForestClassifier 의 예측 정확도는 0.846


In [30]:
from sklearn.ensemble import RandomForestClassifier

model_md1 = RandomForestClassifier(max_depth = 1) # 모델을 객체에 할당
model_md10 = RandomForestClassifier(max_depth = 10)

model_md1.fit(X_train, y_train) # 모델 학습
model_md10.fit(X_train, y_train) 

pred_md1 = model_md1.predict(X_val)
pred_md10 = model_md10.predict(X_val)

print('RF max_depth = 1 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_md1),3)) # 정확도 확인
print('RF max_depth = 10 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_md10),3)) # 정확도 확인

RF max_depth = 1 의 예측 정확도는 0.762
RF max_depth = 10 의 예측 정확도는 0.85


In [31]:
from sklearn.ensemble import RandomForestClassifier

model_ne1 = RandomForestClassifier(n_estimators = 1) # 모델을 객체에 할당
model_ne200 = RandomForestClassifier(n_estimators = 200)

model_ne1.fit(X_train, y_train) # 모델 학습
model_ne200.fit(X_train, y_train) 

pred_ne1 = model_ne1.predict(X_val)
pred_ne200 = model_ne200.predict(X_val)

print('RF n_estimators = 1 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_ne1),3)) # 정확도 확인
print('RF n_estimators = 200 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_ne200),3)) # 정확도 확인

RF n_estimators = 1 의 예측 정확도는 0.788
RF n_estimators = 200 의 예측 정확도는 0.844


In [32]:
from sklearn.ensemble import RandomForestClassifier

model_mf1 = RandomForestClassifier(max_features = 1) # 모델을 객체에 할당
model_mf3 = RandomForestClassifier(max_features = 3)

model_mf1.fit(X_train, y_train) # 모델 학습
model_mf3.fit(X_train, y_train) 

pred_mf1 = model_mf1.predict(X_val)
pred_mf3 = model_mf3.predict(X_val)

print('RF n_estimators = 1 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_mf1),3)) # 정확도 확인
print('RF n_estimators = 3 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_mf3),3)) # 정확도 확인

RF n_estimators = 1 의 예측 정확도는 0.841
RF n_estimators = 3 의 예측 정확도는 0.843


In [33]:
model_2 = RandomForestClassifier(max_depth = 10, n_estimators = 200, max_features = 3) # 최종 모델
model_2.fit(X_train, y_train) 
y_pred_2 = model_2.predict(X_val)
print(y_pred_2)
print('Tuned RandomForestClassifier 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred_2),3)) # 정확도 확인

[0 0 0 ... 1 0 1]
Tuned RandomForestClassifier 의 예측 정확도는 0.85


In [34]:
test = pd.read_csv('/content/drive/MyDrive/깃허브/인구데이터기반소득예측/test.csv')
test = label_encoder(test, make_label_map(test))
test = test.drop('id', axis=1)
test.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,47,0,304133,0,10,0,0,0,0,0,0,0,45,0
1,34,1,154227,0,10,1,1,1,0,0,0,0,75,0
2,31,2,158291,1,13,1,2,1,0,0,8614,0,40,0
3,28,0,183155,1,13,0,3,0,0,0,0,0,55,0
4,54,2,182543,0,10,2,4,2,0,1,0,0,40,1


In [35]:
X_train= data.drop(['target'], axis=1) #전체 training 데이터에서 독립변수 추출
y_train = data.target #전체 training 데이터에서 라벨 추출

In [36]:
model = RandomForestClassifier(max_depth = 10, n_estimators = 200, max_features = 3) #앞서 튜닝한 하이퍼 파라미터 max_depth, n_estimators, max_features
model.fit(X_train, y_train) 
y_pred = model.predict(test)

In [37]:
submission = pd.read_csv('/content/drive/MyDrive/깃허브/인구데이터기반소득예측/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [38]:
submission['target'] = y_pred

# 데이터가 잘 들어갔는지 확인합니다
submission

,id,target
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
...,...,...
15076,15076,1
15077,15077,1
15078,15078,0
15079,15079,0


In [39]:
submission.to_csv('submit_2.csv', index=False)